#  Terminology Project - M2 TAL - 24/11/2020

## Axel Didier - Pierre Goncalves - M2 TAL

### REQUIREMENTS

In [1]:
#Packages : os, spacy, flair

import sys

#!{sys.executable} -m pip install flair
#!{sys.executable} -m pip install spacy


## Rule-based identification system

In [2]:
import os
import re
import spacy

nlp = spacy.load("en_core_web_sm")

### Create a .txt lexicon with a .tsv as input

In [3]:
def clean_term():
    f = open("TermsTSV/MOBA-en.tsv", "r")
    raw = f.read()
    
    #get term raw of the tsv file
    raw_terms = re.findall('\:\s(.*[a-z])', raw)

    result = []
    #loop on every terms
    for item in raw_terms:
        lemmatized_term = ''
        #loop on every token of the term
        doc = nlp(item)
        for token in doc:
            lemmatized_term += token.lemma_ + ' '
            
        print(lemmatized_term[:-1]) #-1 to remove the last white space
        
        #to avoid doublon
        if lemmatized_term not in result:
            result.append(lemmatized_term)
            
    #write all terms in a file
    with open('MOBA-en-lexicon.txt', 'w') as f:
        for item in result:
            f.write(item+"\n")
            
clean_term()

moba
avatar
game avatar
moba game
toxic player
toxic and typical player
toxic and normal player
typical and toxic player
player skill
player base skill
player skill decomposition
player skill composition
player skill evaluation
player skill formation
player skill rating
player skill analysis
team composition
dota
dota
creeps
team formation
league of legend
online game
hot
dot
typical player
gameplay
enemy team
team fight
defense
in - game
riot game
blue team
red and blue team
enemy hero
tactical analysis
late game
player behavior
prize
champion attribute
opposite team
play style
user report
red team
red and blue team
experienced role
inexperienced role
minion
mana
creep
easy mode
red side
blue side
normal player
novice player
lol
lol player
game match
enemy base
noob
mage
matchmaking
skill rating
win rate
low winning rate
enemy tower
hard mode
high skill player
target selection
gamer
opponent team
team role
amount of gold
skillful
gameplay style
neutral creep
rating system
elo rating s

### Get functions (lexicon, article.txt, list of terms in the lexicon)

In [4]:
def get_lexicon():
    f = open("MOBA-en-lexicon.txt", "r")
    lexicon = f.read().splitlines() #use splitlines to not take '\n' char
    #return a list of splited list (to ease the further process)
    return [item.split() for item in lexicon]

def get_doc(file_name):
    f = open(file_name, "r", encoding="utf8")
    text = f.read()
    print(file_name, '|| Text length: ',len(text))
    return text

def get_possible_term(lexicon, word):
    possible_term = []
    for item in lexicon:
        if word == item[0]:
            possible_term.append(item)
    return possible_term

lexicon = get_lexicon()

In [5]:
# Main function to create an IOB annotated .txt file
# Manage one joker word / don't manage syntactic variants
def annotate_doc(txt, f):
    
    txt_pos=0
    while txt_pos < len(txt):#in range(0, len(txt)):
        word = str(txt[txt_pos].lemma_)
        #print(">>", word)
        
        possible_terms = get_possible_term(lexicon, word)
        match = possible_terms
        #print("Possible: ",match)
        
        # If there is possible_terms
        if possible_terms is not None:
            
            to_remove = []
            joker = True
            joker_pos = None
            joker_word = None
            # loop on them
            for term in possible_terms:
                #print(">", term)
                
                # Process for each term word
                for term_pos in range(0, len(term)):
                
                    #check if we dont go out the text length
                    if txt_pos+term_pos < len(txt):
                        #check if word term are not the same of the text word
                        if term[term_pos] != str(txt[txt_pos+term_pos].lemma_):
                            #if not the same but the next is the same, means that still have to process (rule)
                            if term[term_pos] == str(txt[txt_pos+term_pos + 1].lemma_) and joker is True:
                                #joker to False because we can only do that one time
                                joker = False
                                joker_pos = term_pos
                                joker_word = str(txt[txt_pos+term_pos])
                                break
                            else:
                                to_remove.append(term)
                                break
                    else:
                        to_remove.append(term)
                        break
            
            #remove all the terms that didn't match
            for t in to_remove:
                match.remove(t)
                
            # if there is a match
            if len(match)>0:
                #take the longuest
                real_match = max(match, key=len)
                #print("Real Match: ", real_match)
                
                # and jump to the next word
                if joker is False: 
                    f.write(str(txt[txt_pos]) + " B\n")
                    #print("Write: " + str(txt[txt_pos]) + " B\n")
                    for i in range(1, len(real_match)+1):
                        if joker_pos == i:
                            f.write(joker_word + " O\n")
                            #print("Write: " + joker_word + " O\n")
                        else: 
                            f.write(str(txt[txt_pos+i]) + " I\n")
                            #print("Write: " + str(txt[txt_pos+i]) + " I\n")
                    txt_pos += len(real_match)+1
                        
                else: 
                    
                    f.write(str(txt[txt_pos]) + " B\n")
                    #print("Write: " + str(txt[txt_pos]) + " B\n")
                    for i in range(1, len(real_match)):
                        f.write(str(txt[txt_pos+i]) + " I\n")
                        #print("Write: " + str(txt[txt_pos+i]) + " I\n")
                    txt_pos += len(real_match)
            else:
                if word is not "\n":
                    f.write(str(txt[txt_pos])+" O\n")
                    #print("Write: " + str(txt[txt_pos])+" O\n")
                txt_pos += 1
                if word is ".": f.write("\n")

### Creation of the annotated Corpus with IOB (.txt)
#### COMMENT the last line if you don't want to run it (already train.txt already exist) 

In [6]:
# Create TRAIN.TXT file
def train():
    f = open('Corpus/train.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[0:20]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want
train()

doc_01.txt
Articles/txt/doc_01.txt || Text length:  17603
doc_02.txt
Articles/txt/doc_02.txt || Text length:  29695
doc_03.txt
Articles/txt/doc_03.txt || Text length:  70327
doc_04.txt
Articles/txt/doc_04.txt || Text length:  22563
doc_05.txt
Articles/txt/doc_05.txt || Text length:  23758
doc_06.txt
Articles/txt/doc_06.txt || Text length:  32981
doc_07.txt
Articles/txt/doc_07.txt || Text length:  18915
doc_08.txt
Articles/txt/doc_08.txt || Text length:  45445
doc_09.txt
Articles/txt/doc_09.txt || Text length:  42509
doc_10.txt
Articles/txt/doc_10.txt || Text length:  43999
doc_11.txt
Articles/txt/doc_11.txt || Text length:  54214
doc_12.txt
Articles/txt/doc_12.txt || Text length:  78258
doc_13.txt
Articles/txt/doc_13.txt || Text length:  27403
doc_14.txt
Articles/txt/doc_14.txt || Text length:  25424
doc_15.txt
Articles/txt/doc_15.txt || Text length:  38869
doc_16.txt
Articles/txt/doc_16.txt || Text length:  38604
doc_17.txt
Articles/txt/doc_17.txt || Text length:  63251
doc_18.txt
Art

In [7]:
# Create TEST.TXT file
def test():
    f = open('Corpus/test.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[20:25]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want 
test()

doc_21.txt
Articles/txt/doc_21.txt || Text length:  43704
doc_22.txt
Articles/txt/doc_22.txt || Text length:  20314
doc_23.txt
Articles/txt/doc_23.txt || Text length:  18202
doc_24.txt
Articles/txt/doc_24.txt || Text length:  42682
doc_25.txt
Articles/txt/doc_25.txt || Text length:  51958


In [8]:
# Create DEV.TXT file
def dev():
    f = open('Corpus/dev.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[25:30]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want 
dev()

doc_26.txt
Articles/txt/doc_26.txt || Text length:  60221
doc_27.txt
Articles/txt/doc_27.txt || Text length:  51510
doc_28.txt
Articles/txt/doc_28.txt || Text length:  92987
doc_29.txt
Articles/txt/doc_29.txt || Text length:  55843
doc_30.txt
Articles/txt/doc_30.txt || Text length:  19363


## Sequence Tagger 

### Reading Your Own Sequence Labeling Dataset using FLAIR

In [9]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'iob'}

# this is the folder in which train, test and dev files reside
data_folder = './Corpus/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

2020-11-20 00:21:39,887 Reading data from Corpus
2020-11-20 00:21:39,888 Train: Corpus\train.txt
2020-11-20 00:21:39,888 Dev: Corpus\dev.txt
2020-11-20 00:21:39,889 Test: Corpus\test.txt


In [10]:
len(corpus.train)

8001

In [11]:
print(corpus.train[800].to_tagged_string('iob'))

When a player <B> ’s hit <B> points <I> reach 0 , he dies , a kill <B> ( with associated gold <B> and experience <B> ) is awarded to his killers <B> , and he must wait a certain period of time <B> to “ respawn <B> ” and return to the fight <B> .


### Training a Sequence Labeling Model

In [12]:
from flair.data import Corpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings

# 1. get the corpus
print(corpus)

# 2. what tag do we want to predict?
tag_type = 'iob'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-pos',
              embeddings_storage_mode='gpu',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

Corpus: 8001 train + 2092 dev + 1577 test sentences
Dictionary with 6 tags: <unk>, O, B, I, <START>, <STOP>
2020-11-20 00:21:49,461 ----------------------------------------------------------------------------------------------------
2020-11-20 00:21:49,462 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-11-20 00:21:49,462 ----------------------------------------------------------------------------------------------------
2020-11-20 00:21:49,463 Corpus: "Corpus: 8001 train + 2092 dev + 1577 test sentences"
2020-11-20 00:21:49,464 ----------------------------------------------------------------

2020-11-20 00:24:07,789 epoch 5 - iter 75/251 - loss 3.90840168 - samples/sec: 316.69 - lr: 0.100000
2020-11-20 00:24:10,340 epoch 5 - iter 100/251 - loss 3.88833683 - samples/sec: 313.63 - lr: 0.100000
2020-11-20 00:24:12,645 epoch 5 - iter 125/251 - loss 3.86549913 - samples/sec: 347.37 - lr: 0.100000
2020-11-20 00:24:15,081 epoch 5 - iter 150/251 - loss 3.86519271 - samples/sec: 328.60 - lr: 0.100000
2020-11-20 00:24:17,683 epoch 5 - iter 175/251 - loss 3.87444750 - samples/sec: 307.70 - lr: 0.100000
2020-11-20 00:24:20,057 epoch 5 - iter 200/251 - loss 3.89834839 - samples/sec: 337.10 - lr: 0.100000
2020-11-20 00:24:22,485 epoch 5 - iter 225/251 - loss 3.90364290 - samples/sec: 329.64 - lr: 0.100000
2020-11-20 00:24:25,168 epoch 5 - iter 250/251 - loss 3.88807444 - samples/sec: 298.28 - lr: 0.100000
2020-11-20 00:24:25,187 ----------------------------------------------------------------------------------------------------
2020-11-20 00:24:25,187 EPOCH 5 done: loss 3.8734 - lr 0.100

2020-11-20 00:27:00,139 EPOCH 10 done: loss 3.4308 - lr 0.1000000
2020-11-20 00:27:03,698 DEV : loss 2.6863417625427246 - score 0.9633
2020-11-20 00:27:03,711 BAD EPOCHS (no improvement): 0
saving best model
2020-11-20 00:27:09,485 ----------------------------------------------------------------------------------------------------
2020-11-20 00:27:09,486 Testing using best model ...
2020-11-20 00:27:09,487 loading file resources\taggers\example-pos\best-model.pt
2020-11-20 00:27:13,092 	0.9592
2020-11-20 00:27:13,092 
Results:
- F-score (micro): 0.9592
- F-score (macro): 0.7061
- Accuracy (incl. no class): 0.9592

By class:
              precision    recall  f1-score   support

           O     0.9704    0.9890    0.9796     32082
           B     0.7885    0.6591    0.7180      2235
           I     0.6989    0.3009    0.4207       432

    accuracy                         0.9592     34749
   macro avg     0.8193    0.6497    0.7061     34749
weighted avg     0.9554    0.9592    0.955

{'test_score': 0.9592,
 'dev_score_history': [0.9511,
  0.9528,
  0.953,
  0.9547,
  0.9542,
  0.9553,
  0.9577,
  0.96,
  0.9594,
  0.9633],
 'train_loss_history': [5.906162380697243,
  4.467967755291092,
  4.166589154665214,
  4.047764904470558,
  3.8733545214056493,
  3.7903423698774845,
  3.679923255605052,
  3.5557636374021433,
  3.481985113060332,
  3.430843458707589],
 'dev_loss_history': [4.2321343421936035,
  3.9105308055877686,
  3.4715867042541504,
  3.283658504486084,
  3.2400383949279785,
  3.3694121837615967,
  2.9217662811279297,
  2.8845086097717285,
  2.8093247413635254,
  2.6863417625427246]}

### Example of the two identification systems on a text of your targeted domain

#### rule-based identification system created from scratch

In [13]:
def docRB():
    f = open('resources/doc-RB1.txt', 'w', encoding="utf8")
    doc = nlp(get_doc("Articles/txt/file_01.txt"))
    annotate_doc(doc, f)
    f.close()
    
docRB()

Articles/txt/doc_01.txt || Text length:  17603


In [14]:
doctest2 = get_doc("resources/doc-RB1.txt")
doctest2 = doctest2.replace("\n", " ")
doctest2 = doctest2.replace(" O ", " ")
doctest2 = doctest2.replace(" B ", " <B> ")
doctest2 = doctest2.replace(" I ", " <I> ")
doctest2 = doctest2.replace(" .", ".")
doctest2

resources/tagtest/doc-RB1.txt || Text length:  25470


'HAL <I> d : hal-01771253 https://hal.inria.fr/hal-01771253 Submitted on 19 Apr 2018 HAL is a multi - disciplinary open <B> access archive for the deposit and dissemination of scientific research documents , whether they are published or not.  The documents may come from teaching and research institutions in France or abroad , or from public or private research centers.  L’archive ouverte pluridisciplinaire HAL , est destinée au dépôt et à la diffusion de documents scientifiques de niveau recherche , publiés ou non , émanant des établissements d’enseignement et de recherche français ou étrangers , des laboratoires publics ou privés.  Distributed under a Creative Commons Attribution| 4.0 International License An Analysis of DOTA2 Using Game Refinement Measure Long Zuo , Shuo Xiong , Hiroyuki Iida To cite this version : Long Zuo , Shuo Xiong , Hiroyuki Iida.  An Analysis of DOTA2 Using Game Refinement Measure.  16th International Conference on Entertainment Computing ( ICEC ) , Sep 2017 

#### Sequence tagger trained model from flair

In [15]:
from flair.data import Sentence

# load the model you trained
model = SequenceTagger.load('resources/taggers/example-pos/final-model.pt')

# create example sentence
doctest = get_doc("Articles/txt/file_01.txt")
sentence = Sentence(doctest)

# predict tags and print
model.predict(sentence)

doctest1 = sentence.to_tagged_string()
doctest1

2020-11-20 00:27:13,580 loading file resources/taggers/example-pos/final-model.pt
Articles/txt/doc_01.txt || Text length:  17603


'HAL Id : hal-01771253 https :// hal.inria.fr / hal-01771253 Submitted on 19 Apr 2018 HAL is a multi-disciplinary open access archive for the deposit and dissemination of scientific research documents , whether they are published or not . The documents may come from teaching and research institutions in France or abroad , or from public or private research centers . L’archive ouverte pluridisciplinaire HAL , est destinée au dépôt et à la diffusion de documents scientifiques de niveau recherche , publiés ou non , émanant des établissements d’enseignement et de recherche français ou étrangers , des laboratoires publics ou privés . Distributed under a Creative Commons Attribution | 4.0 International License An Analysis of DOTA2 Using Game <B> Refinement Measure Long Zuo , Shuo Xiong , Hiroyuki Iida To cite this version : Long Zuo , Shuo Xiong , Hiroyuki Iida . An Analysis of DOTA2 Using Game <B> Refinement Measure . 16th International Conference on Entertainment Computing ( ICEC ) , Sep 2